## Overview
This notebook is used to generate various test vectors to support FALCON Digital Signal Processing unit tests. Vectors include: Finite Impulse Response (FIR) filter coefficients, raw complex input data, and raw complex output data.

## Constants
This notebook supports configurable behavior through environment variables, which in turn modify the notebook-level constants specified here. See [documentation](https://pythonhosted.org/jupyter_runner/) for an example of how to run the notebook with various environment variable configurations (excerpt shown here).

#### How-to: Run notebook with multiple sets of parameters
Create a file with multiple set of parameters, one set of parameters per line.

Example file containing 2 sets of 3 parameters:

VAR1=VAL1 VAR2=VAL2 VAR3=VAL3
VAR1=VAL5 VAR2=VAL18 VAR3='VAL42 with space'
Then run jupyter-run specifying the path to my_parameter_file just created:

`jupyter-run --parameter-file=my_parameter_file notebook.ipynb`

By default, the process creates output files notebook_1.html and notebook_2.html in current directory.

## Python Imports

In [ ]:
from fractions import Fraction
import logging
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
import os
from scipy import signal

In [ ]:
FREQ_SHIFT_STR = os.environ.get('FREQ_SHIFT', '0')
INPUT_SAMPLE_RATE_STR = os.environ.get('INPUT_SAMPLE_RATE', '1000000')
OUTPUT_SAMPLE_RATE_STR = os.environ.get('OUTPUT_SAMPLE_RATE', '100000')
NUM_SAMPLES_STR = os.environ.get('NUM_OUTPUT_SAMPLES', '10000')
SHOW_GRAPHS_FLAG_STR = os.environ.get('SHOW_GRAPHS', '0')
VERBOSE_LOGGING_STR = os.environ.get('VERBOSE', '0')

# convert from the string environment variables to other, more convenient, representations
FREQ_SHIFT = float(FREQ_SHIFT_STR)
INPUT_SAMPLE_RATE = float(INPUT_SAMPLE_RATE_STR)
OUTPUT_SAMPLE_RATE = float(OUTPUT_SAMPLE_RATE_STR)
NUM_SAMPLES = int(NUM_SAMPLES_STR)

SHOW_GRAPHS_FLAG = True
if SHOW_GRAPHS_FLAG_STR is not "1":
    SHOW_GRAPHS_FLAG = False
    
VERBOSE_LOGGING = True
if VERBOSE_LOGGING_STR is not "1":
    VERBOSE_LOGGING = False

## Logging
Use the Python *logging* module for basic logging. Configure the logging level based on the provided command-line arguments.

In [ ]:
logging_level = logging.INFO
if VERBOSE_LOGGING:
    logging_level = logging.DEBUG
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging_level)

## Design a Low Pass Filter
The objective is to use the *upfirdn* function so we need to convert the decimal resampling ratio into a fraction. From there, we can design a Finite Impulse Response (FIR) filter that meets our needs.

See the following excerpt from **Boaz Porat, A Course in Digital Signal Processing**:
> "The low-pass filter performs both interpolation of the expanded signal
    and antialiasing. If the sample rate is to be increased, then p > q. THe
    low pass filter should then have a cutoff frequency of pi/p. If the
    sampling rate is to be decreased, then p < q. The low-pass filter should
    then have a cutoff frequency pi/q. IN this case the filter will
    eliminate a part of the signals frequency contents of the original
    bandwidth if its original bandwidth is higher than pip/q. Thus, the
    sampling rate conversion filter should always have a cutoff frequency
    of pi/max{p,q}."

In [ ]:
logging.info("Input Sample Rate: %d Hz" % (INPUT_SAMPLE_RATE))
logging.info("Output Sample Rate: %d Hz" % (OUTPUT_SAMPLE_RATE))

resampling_ratio = OUTPUT_SAMPLE_RATE / INPUT_SAMPLE_RATE
logging.info("Resampling Ratio %f" % (resampling_ratio))
ratio = Fraction(resampling_ratio).limit_denominator()
p = ratio.numerator
q = ratio.denominator
pqmax = max(p, q)
logging.info("Resampling Ratio: %d/%d, pqmax=%d" % (p, q, pqmax))

# cutoff frequency of the lowpass filter at the high (upsampled) rate
cutoff_freq = 1 / 2 / pqmax
filter_order = 2 * 10 * pqmax + 1
filter_delay = (filter_order - 1) / p / 2

logging.info("Cutoff Freq: %f, Filter Order: %u Filter Delay: %u" % (cutoff_freq, filter_order, filter_delay))

filter_coeffs = signal.firls(filter_order, [0, 2 * cutoff_freq, 2 * cutoff_freq, 1], [1, 1, 0, 0])
filter_coeffs = filter_coeffs * signal.kaiser(filter_order, beta=5)

# TODO write filter coefficients to file

# visualize the filter response
w, h = signal.freqz(filter_coeffs)
fig = plt.figure()
plt.title('Digital filter frequency response')
ax1 = fig.add_subplot(111)

plt.plot(w, 20 * np.log10(abs(h)), 'b')
plt.ylabel('Amplitude [dB]', color='b')
plt.xlabel('Frequency [rad/sample]')

ax2 = ax1.twinx()
angles = np.unwrap(np.angle(h))
plt.plot(w, angles, 'g')
plt.ylabel('Angle (radians)', color='g')
plt.grid()
plt.axis('tight')
plt.show()

## Create a Complex Sinusoid

In [ ]:
# for fractional upsampling put the test tone closer to DC so that there
#  is more definition to the graphed original signal
us_test_tone_freq = (INPUT_SAMPLE_RATE * 0.3) / 2

# for fractional downsampling put the test tone at the outer edge of the
#  expected passband in order to verify that the filter is working
#  correctly and does not attenuate the input signal too badly
ds_test_tone_freq = (OUTPUT_SAMPLE_RATE * 0.5) / 2

test_tone_freq = float(min(us_test_tone_freq, ds_test_tone_freq))
test_tone_freq = 1000
logging.info("Test tone freq=%u Hz" % (test_tone_freq))

SIGNAL_SCALE = 1024
t = np.arange(0, NUM_SAMPLES)
x = SIGNAL_SCALE * np.exp(1j * 2 * np.pi * t * (test_tone_freq/INPUT_SAMPLE_RATE))

# now frequency shift by the requested amount
fs_vec = np.exp(1j * 2 * np.pi * t * (FREQ_SHIFT/INPUT_SAMPLE_RATE))
x = x * fs_vec

# resample using the computed values
y = signal.upfirdn(filter_coeffs, x, p, q)

### Time Series Plot

In [ ]:
resampled_t = (np.arange(0, NUM_SAMPLES * p / q) * (p / q)) - filter_delay

plt.figure(figsize=(10, 8))
plt.subplot(2,1,1)
plt.plot(t, x.real, 'b')
plt.plot(resampled_t, y.real, '-xr')
plt.xlabel('t')
plt.ylabel('Re x(t)')
plt.title(r'Real part of  $x(t)=e^{j 2 \pi f t}$');

plt.subplot(2,1,2)
plt.plot(t, x.imag, 'b')
plt.xlabel('t')
plt.ylabel('Im x(t)')
plt.title(r'Imaginary part of  $x(t)=e^{j 2 \pi f t}$');
plt.tight_layout()
plt.show()

### Spectrogram Plot

In [ ]:
Pxx, freqs, bins, im = plt.specgram(y, NFFT=256, Fs=INPUT_SAMPLE_RATE)
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.show()